In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import plotly
import plotly.graph_objs as go
import plotly.io as pio

In [14]:
# Load dataframes from csv files
census_df = pd.read_csv('df_census.csv', index_col=0)
ec_df = pd.read_csv('df_ec.csv', index_col=0)
election_df = pd.read_csv('df_election.csv', index_col=0)
abbrev_df = pd.read_csv('df_abbrev.csv', index_col=0)

In [39]:
print(census_df.loc["United States", census_df.columns[0]])
print(sum(census_df.loc["Alabama":"Wyoming", "HC01_VC03"]))
print(type(census_df.loc["Alabama":"Wyoming", "HC01_VC03"]))
census_df.loc[:, "HC01_VC03"].dtype
ec_df["Number of Electoral Votes"].dtype

325719178
325719178
<class 'pandas.core.series.Series'>


dtype('int64')

In [48]:
# Calculate Average voting power (EV share PER person)
avg_vp = ec_df["Number of Electoral Votes"].sum() / float(census_df.loc["United States", "HC01_VC03"])
print('Average share of an electoral vote for an individual:\n{:.2e}'
      .format(avg_vp))

# Store inverse
people_per_ev = 1.0/avg_vp
print('\nAverage amount of people per electoral vote:\n{:.2f}'.format(people_per_ev))

Average share of an electoral vote for an individual:
1.65e-06

Average amount of people per electoral vote:
605425.98


### I) Electoral Vote Apportionment Bar Chart

blahblah

In [64]:
ec_df.loc['Alabama']['Number of Electoral Votes']

9

In [199]:
# Get list of states in decreasing order of population
states = sorted(list(census_df.index[1:]), 
                key=lambda state: census_df.loc[state, 'HC01_VC03'],
                reverse=True)

state_pop = go.Bar(
    x=list(range(51)),#states,
    y=[census_df.loc[state, 'HC01_VC03'] for state in states],
    name='Population',
    marker=dict(
        color='#C2EFEB',
        #opacity=,
    )
)

ev_color = '#6EA4BF'
state_ev = go.Bar(
    x=list(range(51)),#states,
    y=[ec_df.loc[state, 'Number of Electoral Votes'] for state in states],
    name='Electoral Votes',
    yaxis='y2',
    #offset=4,
    marker=dict(
        color=ev_color,
        #opacity=,
    ),
    width=0.2,
)

# Hacky way to get grouped bars on multiple y-axes. Not working.
dummies = [
    go.Bar(
        x=['Alabama'], 
        y=[0],
        yaxis='y',
        name='y dummy', 
        hoverinfo='none', 
        showlegend=False
    ), 
    go.Bar(
        x=['Alabama'],
        y=[0],
        yaxis='y2', 
        name='y2 dummy', 
        hoverinfo='none', 
        showlegend=False
    ),
]

b_top = census_df.loc['California', 'HC01_VC03']
b_bot = 33300000
b_x = 1.2
b_size = 1.0
bracket = go.Scatter(
    x=[b_x, b_x+b_size, b_x+b_size, b_x],
    y=[b_bot, b_bot, b_top, b_top],
    fill=None,
    mode='lines',
    line=dict(
        width=1.0,
        color='black',
    ),
)

data = [state_pop, state_ev] + [bracket] #+ dummies

layout = go.Layout(
    title=dict(   
        text='State populations and electoral votes are NOT proportional',
        font=dict(
            family='verdana',
            size=18,
            color=None,
        ),
        #xref='paper',
        yref='paper',
        y=1.0,
        yanchor='bottom',
        #xanchor='right',
        pad=dict( # unit: pixels
            b=20,
        ),
    ),
    width=900,
    height=500,
    #barmode='group',
    showlegend=False,
    xaxis=dict(
        tickfont=dict(
            size=10,
        ),
        tickangle=-65,
        #automargin=True,
        tickmode='array',
        tickvals=list(range(51)),
        ticktext=states[:states.index('District of Columbia')] \
                 + ['D.C.'] + states[states.index('District of Columbia')+1:] 
    ),
    yaxis=dict(
        title='Population',
        range=[0,40000000],
        tickfont=dict(
            size=12,
        ),
        rangemode='tozero',
    ),
    yaxis2=dict(
        title='Electoral Votes',
        range=[0,40000000/float(people_per_ev)],
        color=ev_color,
#         titlefont=dict(
#             color=ev_color,
#         ),
        tickfont=dict(
            size=12,
        ),
        overlaying='y',
        side='right',
        rangemode='tozero',
        zeroline=False,
    )
)

notes = [dict(
    text='<i>This gap represents population<br>' +\
         'not represented by electoral votes</i>',
    font=dict(
        family=None,
        size=12,
        color='black',
    ),
    x=b_x + b_size,
    y=(b_top + b_bot)/2.0,
    xref='x',
    yref='y',
    align='left',
    xanchor='left',
    yanchor='middle',
    showarrow=True,
    arrowcolor='black',
    arrowwidth=1.2,
    arrowhead=0,
    arrowsize=1,
    startstandoff=3,
    axref='x',
    ayref='y',
    ax=6,
    ay=(b_top + b_bot)/2.0,
    xshift=0,
)]

layout.update(annotations=notes)

fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

In [201]:
pio.write_image(fig, 'images/ec_bars01.png', format='png', scale=3.0)

In [202]:
# Get list of states in decreasing order of population
states = sorted(list(census_df.index[1:]), 
                key=lambda state: census_df.loc[state, 'HC01_VC03'],
                reverse=True)

bar_width = 0.36
state_pop = go.Bar(
    x=list(range(51)),#states,
    y=[census_df.loc[state, 'HC01_VC03'] for state in states],
    name='Population',
    marker=dict(
        color='#C2EFEB',
        #opacity=,
    ),
    width=bar_width
)

ev_color = '#6EA4BF'
state_ev = go.Bar(
    x=list(range(51)),#states,
    y=[ec_df.loc[state, 'Number of Electoral Votes'] for state in states],
    name='Electoral Votes',
    yaxis='y2',
    marker=dict(
        color=ev_color,
        #opacity=,
    ),
    width=bar_width,
    offset=bar_width/2.0,
)

b_top = census_df.loc['California', 'HC01_VC03']
b_bot = 33300000
b_x = 1.2
b_size = 1.0
bracket = go.Scatter(
    x=[b_x, b_x+b_size, b_x+b_size, b_x],
    y=[b_bot, b_bot, b_top, b_top],
    fill=None,
    mode='lines',
    line=dict(
        width=1.0,
        color='black',
    ),
)

data = [state_pop, state_ev] + [bracket]

layout = go.Layout(
    title=dict(   
        text='State populations and electoral votes are NOT proportional',
        font=dict(
            family='verdana',
            size=18,
            color=None,
        ),
        #xref='paper',
        yref='paper',
        y=1.0,
        yanchor='bottom',
        #xanchor='right',
        pad=dict( # unit: pixels
            b=20,
        ),
    ),
    width=900,
    height=500,
    barmode='group',
    showlegend=False,
    xaxis=dict(
        tickfont=dict(
            size=10,
        ),
        tickangle=-65,
        #automargin=True,
        tickmode='array',
        tickvals=list(range(51)),
        ticktext=states[:states.index('District of Columbia')] \
                 + ['D.C.'] + states[states.index('District of Columbia')+1:] 
    ),
    yaxis=dict(
        title='Population',
        range=[0,40000000],
        tickfont=dict(
            size=12,
        ),
        rangemode='tozero',
    ),
    yaxis2=dict(
        title='Electoral Votes',
        range=[0,40000000/float(people_per_ev)],
        color=ev_color,
#         titlefont=dict(
#             color=ev_color,
#         ),
        tickfont=dict(
            size=12,
        ),
        overlaying='y',
        side='right',
        rangemode='tozero',
        zeroline=False,
    )
)

notes = [dict(
    text='<i>This gap represents population<br>' +\
         'not represented by electoral votes</i>',
    font=dict(
        family=None,
        size=12,
        color='black',
    ),
    x=b_x + b_size,
    y=(b_top + b_bot)/2.0,
    xref='x',
    yref='y',
    align='left',
    xanchor='left',
    yanchor='middle',
    showarrow=True,
    arrowcolor='black',
    arrowwidth=1.2,
    arrowhead=0,
    arrowsize=1,
    startstandoff=3,
    axref='x',
    ayref='y',
    ax=6,
    ay=(b_top + b_bot)/2.0,
    xshift=0,
)]

layout.update(annotations=notes)

fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

In [203]:
pio.write_image(fig, 'images/ec_bars02.png', format='png', scale=3.0)

In [117]:
[0:10]

SyntaxError: invalid syntax (<ipython-input-117-2f292e6bc2b5>, line 1)